In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.pipeline import make_pipeline
from collections import Counter
import glob
import tensorflow as tf
import math
from keras.utils import to_categorical
import pandas as pd
import os
import soundfile as sf
import numpy as np
import torch


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00


In [ ]:
pip install torch torchvision


In [ ]:
# from transformers import Wav2Vec2Model, Wav2Vec2Tokenizer

# tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

from transformers import Wav2Vec2Model, Wav2Vec2Tokenizer

# Load the tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-italian")

model = Wav2Vec2Model.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-italian")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:792: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


In [ ]:
df_labels = pd.read_csv('/content/drive/MyDrive/Android/audio_data.csv')
df_labels

,speaker_id,gender,age,education,audio_clip,depression,full_speaker_id
0,65_P,0,41,3,65_PF41_3_4,1,65_PF41_3
1,65_P,0,41,3,65_PF41_3_2,1,65_PF41_3
2,65_P,0,41,3,65_PF41_3_1,1,65_PF41_3
3,65_P,0,41,3,65_PF41_3_3,1,65_PF41_3
4,68_P,0,43,3,68_PF43_3_1,1,68_PF43_3
...,...,...,...,...,...,...,...
869,02_C,1,57,2,02_CM57_2_10,0,02_CM57_2
870,03_P,0,66,3,03_PF66_3_3,1,03_PF66_3
871,03_P,0,66,3,03_PF66_3_4,1,03_PF66_3
872,03_P,0,66,3,03_PF66_3_2,1,03_PF66_3


In [ ]:
# Function to extract features from audio
def extract_features_from_audio(file_path):
    # Read audio file
    input_audio, _ = sf.read(file_path)
    # Tokenize audio file
    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values
    # Extract features using Wav2Vec
    with torch.no_grad():
        features = model(input_values).last_hidden_state
    return features.numpy()

# Directory where audio files are stored
audio_dir = '/content/drive/MyDrive/Android/audio_clip'

# Lists to hold features and labels
audio_features_list = []
label_list = []

# Iterate over each subfolder in the directory
for subdir in os.listdir(audio_dir):
    subdir_path = os.path.join(audio_dir, subdir)

    # Ensure it's a directory and not a stray file
    if os.path.isdir(subdir_path):

        # Iterate over each audio file in the subdirectory
        for audio_filename in os.listdir(subdir_path):
            if audio_filename.endswith('.wav'):
                # Construct full path of audio file
                file_path = os.path.join(subdir_path, audio_filename)
                print(file_path)

                # Extract features from audio file
                features = extract_features_from_audio(file_path)
                audio_features_list.append(features)

               # Remove the .wav extension from the filename for comparison
                audio_filename_without_extension = os.path.splitext(audio_filename)[0]

                # Safely map the associated label using the filename without extension
                label = df_labels[df_labels['audio_clip'] == audio_filename_without_extension].iloc[0]['depression']  # Adjust 'label' column name accordingly
                label_list.append(label)



# Convert lists to numpy arrays for training purposes
import numpy as np
audio_features = np.vstack(audio_features_list)
labels = np.array(label_list)


/content/drive/MyDrive/Basma/Android/audio_clip/65_PF41_3/65_PF41_3_4.wav
/content/drive/MyDrive/Basma/Android/audio_clip/65_PF41_3/65_PF41_3_2.wav
/content/drive/MyDrive/Basma/Android/audio_clip/65_PF41_3/65_PF41_3_1.wav
/content/drive/MyDrive/Basma/Android/audio_clip/65_PF41_3/65_PF41_3_3.wav
/content/drive/MyDrive/Basma/Android/audio_clip/68_PF43_3/68_PF43_3_1.wav
/content/drive/MyDrive/Basma/Android/audio_clip/68_PF43_3/68_PF43_3_2.wav
/content/drive/MyDrive/Basma/Android/audio_clip/68_PF43_3/68_PF43_3_3.wav
/content/drive/MyDrive/Basma/Android/audio_clip/68_PF43_3/68_PF43_3_6.wav
/content/drive/MyDrive/Basma/Android/audio_clip/68_PF43_3/68_PF43_3_5.wav
/content/drive/MyDrive/Basma/Android/audio_clip/68_PF43_3/68_PF43_3_4.wav
/content/drive/MyDrive/Basma/Android/audio_clip/64_PF54_2/64_PF54_2_6.wav
/content/drive/MyDrive/Basma/Android/audio_clip/64_PF54_2/64_PF54_2_1.wav
/content/drive/MyDrive/Basma/Android/audio_clip/64_PF54_2/64_PF54_2_4.wav
/content/drive/MyDrive/Basma/Android/a

In [ ]:
audio_features.shape

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 1. Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(audio_features, labels, test_size=0.2, random_state=42)

# 2. Preprocessing

# Pad sequences to the same length for training data
MAX_SEQUENCE_LENGTH = max([feature.shape[0] for feature in X_train])
X_train_padded = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post', dtype='float32')
X_test_padded = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post', dtype='float32')

# Convert training labels to one-hot encoding
NUM_CLASSES = len(np.unique(labels))
y_train_onehot = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test_onehot = to_categorical(y_test, num_classes=NUM_CLASSES)

# 3. Build the BiLSTM model

input_shape = (MAX_SEQUENCE_LENGTH, X_train_padded.shape[2])
model = Sequential([
    Input(shape=input_shape),
    Masking(mask_value=0.),  # Mask padding values
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 4. Train the model

EPOCHS = 10
BATCH_SIZE = 32

model.fit(X_train_padded, y_train_onehot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test_padded, y_test_onehot))
